In [ ]:
from tkinter import messagebox
from tkinter import filedialog
import tkinter.ttk as ttk
from pathlib import Path
import tkinter as tk
import numpy as np
import threading
import random
import shutil
import glob
import cv2
import os
import re

class Application(tk.Frame):
    
    def __init__(self, master):
        
        super().__init__(master)
        
        self.master.geometry("425x275")
        self.master.minsize(width = 425, height = 275)
        self.master.maxsize(width = 425, height = 275)
        self.master.title("nearest neighbor algorithm")
        
        # rootメインウィンドウのグリッドを 1x1 にする
        self.master.grid_rowconfigure(0, weight = 1)
        self.master.grid_columnconfigure(0, weight = 1)

#────────────── メイン フレーム ─────────────────#
        # メインフレームの作成と設置
        frame = ttk.Frame(self.master)
        frame.grid(row = 0, column = 0, sticky = "nsew", pady = 20)

        # 各種ウィジェットの作成
        label1_frame = ttk.Label(frame, text = "最短距離法による物体認識")
        
        button_exp = ttk.Button(frame, text = "取扱説明書", command = lambda :self.Change_app(frame_exp))
        button_ave = ttk.Button(frame, text = "平均画像生成", command = lambda :self.Change_app(frame_ave))
        button_pro = ttk.Button(frame, text = "最短距離法で確率算出", command = lambda :self.Change_app(frame_pro))
        
        # 各種ウィジェットの設置
        label1_frame.pack()
        button_exp.pack()
        button_ave.pack()
        button_pro.pack()
        
#───────────── 平均画像生成 フレーム ───────────────#
        # アプリフレームの作成と設置
        frame_ave = ttk.Frame(self.master)
        frame_ave.grid(row = 0, column = 0, sticky = "nsew", pady = 20)
        
        self.ave_var = tk.StringVar()

        # 各種ウィジェットの作成
        label1_frame_ave = ttk.Label(frame_ave, text = "平均画像生成")
        label2_frame_ave = ttk.Label(frame_ave, text = "画像フォルダパス")
        label3_frame_ave = ttk.Label(frame_ave, text = "平均画像フォルダ名")
        
        self.entry1_frame_ave = ttk.Entry(frame_ave, textvariable = self.ave_var)
        self.entry2_frame_ave = ttk.Entry(frame_ave)
        self.entry2_frame_ave.insert(0, "Average")
        
        button_select_ave = ttk.Button(frame_ave, text = "参照", command = lambda:self.Select_folder("ave"))
        button_create_ave = ttk.Button(frame_ave, text = "生成", command = lambda:self.Parallel_processing("ave"))
        
        button_change_frame_ave = ttk.Button(frame_ave, text = "メインウィンドウに移動", command = lambda :self.Change_app(frame))
        button_change_frame_ave_exp = ttk.Button(frame_ave, text = "取扱説明書", command = lambda :self.Change_app(frame_ave_exp))
        
        # 各種ウィジェットの設置
        label1_frame_ave.grid(column = 0, row = 0)
        label2_frame_ave.grid(column = 0, row = 1)
        
        button_select_ave.grid(column = 1,row = 2)
        self.entry1_frame_ave.grid(column = 0, row = 2)
        
        label3_frame_ave.grid(column = 0, row = 3)
        self.entry2_frame_ave.grid(column = 0, row = 4)
        button_create_ave.grid(column = 0, row = 5)
        
        button_change_frame_ave.grid(column = 0, row = 6)
        button_change_frame_ave_exp.grid(column = 0, row = 7)
        
#───────────── 最短距離法確率算出フレーム ───────────────#
        # アプリフレームの作成と設置
        frame_pro = ttk.Frame(self.master)
        frame_pro.grid(row = 0, column = 0, sticky = "nsew", pady = 20)
        
        self.pro_var1 = tk.StringVar()
        self.pro_var2 = tk.StringVar()

        # 各種ウィジェットの作成
        label1_frame_pro = ttk.Label(frame_pro, text = "最短距離法確率算出")
        
        label2_frame_pro = ttk.Label(frame_pro, text = "平均画像フォルダパス")
        label3_frame_pro = ttk.Label(frame_pro, text = "検証画像フォルダパス")
        label4_frame_pro = ttk.Label(frame_pro, text = "検証確率フォルダ名")
        label5_frame_pro = ttk.Label(frame_pro, text = "検証回数")
        
        self.entry1_frame_pro = ttk.Entry(frame_pro, textvariable = self.pro_var1)
        self.entry2_frame_pro = ttk.Entry(frame_pro, textvariable = self.pro_var2)
        self.entry3_frame_pro = ttk.Entry(frame_pro)
        self.entry4_frame_pro = ttk.Entry(frame_pro)
   
        button_select_pro1 = ttk.Button(frame_pro, text = "⇐ 参照", command = lambda:self.Select_folder("aveins"))
        button_select_pro2 = ttk.Button(frame_pro, text = "⇐ 参照", command = lambda:self.Select_folder("imgins"))
        
        button_create_pro = ttk.Button(frame_pro, text = "実行", command = lambda:self.Parallel_processing("pro"))
        
        button_change_frame_pro = ttk.Button(frame_pro, text = "メインウィンドウに移動", command = lambda :self.Change_app(frame))
        button_change_frame_pro_exp = ttk.Button(frame_pro, text = "取扱説明書", command = lambda :self.Change_app(frame_pro_exp))
        
        # 各種ウィジェットの設置
        label1_frame_pro.grid(column = 0, row = 0)
        
        label2_frame_pro.grid(column = 0, row = 1)
        self.entry1_frame_pro.grid(column = 0, row = 2)
        button_select_pro1.grid(column = 1,row = 2)
        
        label3_frame_pro.grid(column = 2, row = 1)
        self.entry2_frame_pro.grid(column = 2, row = 2)
        button_select_pro2.grid(column = 3,row = 2)
        
        label4_frame_pro.grid(column = 0, row = 3)
        self.entry3_frame_pro.grid(column = 0, row = 4)
        
        label5_frame_pro.grid(column = 2, row = 3)
        self.entry4_frame_pro.grid(column = 2, row = 4)
        
        button_create_pro.grid(column = 0, row = 5)
        
        button_change_frame_pro_exp.grid(column = 0, row = 6)
        button_change_frame_pro.grid(column = 0, row = 7)
        
#────────────── メイン取扱説明 フレーム ─────────────────#
        # アプリフレームの作成と設置
        frame_exp = ttk.Frame(self.master)
        frame_exp.grid(row = 0, column = 0, sticky = "nsew", pady = 20)

        # 各種ウィジェットの作成
        label1_frame_exp = ttk.Label(frame_exp, text = "使い方")
        label2_frame_exp = ttk.Label(frame_exp, text = "平均画像をお持ちでない方")
        label3_frame_exp = ttk.Label(frame_exp, text = "「平均画像生成」から画像を生成してください")
        label4_frame_exp = ttk.Label(frame_exp, text = "お持ちの方 / 生成された方")
        label5_frame_exp = ttk.Label(frame_exp, text = "「最短距離法確率算出」から確率を算出してください")
        label6_frame_exp = ttk.Label(frame_exp, text = "さらに詳しい説明はそれぞれのページにある「取扱説明書」まで")

        button_change_frame_exp = ttk.Button(frame_exp, text = "メインウィンドウに移動", command = lambda :self.Change_app(frame))

        # 各種ウィジェットの設置
        label1_frame_exp.pack()
        label2_frame_exp.pack()
        label3_frame_exp.pack()
        label4_frame_exp.pack()
        label5_frame_exp.pack()
        label6_frame_exp.pack()
          
        button_change_frame_exp.pack()
        
#────────────── 平均画像 取扱説明 フレーム ─────────────────#
        # アプリフレームの作成と設置
        frame_ave_exp = ttk.Frame(self.master)
        frame_ave_exp.grid(row = 0, column = 0, sticky = "nsew", pady = 20)

        # 各種ウィジェットの作成
        label1_frame_ave_exp = ttk.Label(frame_ave_exp, text = "使い方")
        label2_frame_ave_exp = ttk.Label(frame_ave_exp, text = "画像の入ったフォルダを指定してください")
        label3_frame_ave_exp = ttk.Label(frame_ave_exp, text = "生成された画像が入るフォルダを指定してください")
        label4_frame_ave_exp = ttk.Label(frame_ave_exp, text = "生成される画像名はフォルダ名が入ります")
        
        button_change_frame_exp = ttk.Button(frame_ave_exp, text = "平均画像生成へ", command = lambda :self.Change_app(frame_ave))

        # 各種ウィジェットの設置
        label1_frame_ave_exp.pack()
        label2_frame_ave_exp.pack()
        label3_frame_ave_exp.pack()
        label4_frame_ave_exp.pack()
          
        button_change_frame_exp.pack()
        
#────────────── 確率算出 取扱説明 フレーム ─────────────────#
        # アプリフレームの作成と設置
        frame_pro_exp = ttk.Frame(self.master)
        frame_pro_exp.grid(row = 0, column = 0, sticky = "nsew", pady = 20)

        # 各種ウィジェットの作成
        label1_frame_pro_exp = ttk.Label(frame_pro_exp, text = "使い方")
        label2_frame_pro_exp = ttk.Label(frame_pro_exp, text = "平均画像の入ったフォルダを指定してください")
        label3_frame_pro_exp = ttk.Label(frame_pro_exp, text = "検証画像は平均画像生成に使ってない画像を使用するのが好ましいです")
        label4_frame_pro_exp = ttk.Label(frame_pro_exp, text = "算出方法は検証画像を100枚ランダムで分類し、正しい分類をされた確率を算出します")

        button_change_frame_exp = ttk.Button(frame_pro_exp, text = "確率算出へ", command = lambda :self.Change_app(frame_pro))

        # 各種ウィジェットの設置
        label1_frame_pro_exp.pack()
        label2_frame_pro_exp.pack()
        label3_frame_pro_exp.pack()
        label4_frame_pro_exp.pack()
          
        button_change_frame_exp.pack()
        
        
        # frameを前面にする
        frame.tkraise()
        
    # 画面を切り替える
    def Change_app(self, window):
        window.tkraise()

# ──────────── 共通処理 ──────────── #
    # フォルダ選択
    def Select_folder(self, some):
        
        folder_dir = os.path.abspath(Path().resolve())
        folder_path = filedialog.askdirectory(initialdir = folder_dir)
        
        if some == "ave":
            self.ave_var.set(folder_path)
        elif some == "aveins":
            self.pro_var1.set(folder_path)
        elif some == "imgins":
            self.pro_var2.set(folder_path)
        
    # パスの確認
    def Check_path(self, path):
        
        if path == "":
            messagebox.showerror('エラー', '画像フォルダが指定されていません')
        elif os.path.exists(path) == False:
            messagebox.showerror('エラー', 'パスが間違っています')
        else:
            pass
        
    def Check_num(self, num):
        
        if num == "":
            messagebox.showerror('エラー', 'フォルダ名が指定されていません')
        elif num == "0":
            messagebox.showerror('エラー', '0は指定できません')
        elif num.isdecimal() == False:
            messagebox.showerror('エラー', '整数を入力してください')
        else:
            pass
        
    # フォルダ名が空白かどうかの確認
    def Check_name(self, name):
        
        if name == "":
            messagebox.showerror('エラー', 'フォルダ名が指定されていません')
        else:
            pass
    
    def Parallel_processing(self, divide):
        
        if divide == "ave":
            thread_ave = threading.Thread(target = self.Balance)
            thread_ave.start()
            
        elif divide == "pro":
            thread_pro = threading.Thread(target = self.Path_get)
            thread_pro.start()
    
    # 画像のサイズを取得
    def Minsize(self, image_path):
        
        for i,file in enumerate(os.listdir(image_path)):
            
            img = cv2.imread(image_path + file)
            h, w, c = img.shape

            if i == 0:
                width = w
                height = h
            elif width != height:
                messagebox.showerror('エラー', "画像が正方形ではありません")
                break
            elif width != w:
                messagebox.showerror('エラー', file + "この画像の高さが他と違います")
                break
            elif height != h:
                messagebox.showerror('エラー', file + "この画像の幅が他と違います")
                break
            else:
                return int(h)
    
# ──────────── 平均画像生成 関係 ──────────── #
    # 平均画像生成
    def Balance(self):

        file_path = self.entry1_frame_ave.get() + "/"
        ave_file_name = self.entry2_frame_ave.get()
        
        self.Check_path(file_path)
        self.Check_name(ave_file_name)
        
        
        for k in os.listdir(file_path):
            
            extension = os.path.splitext(os.path.basename(file_path + k))
            
            if extension[1] == ".jpg" or extension[1] == ".png":
                image_path = file_path
            else:
                image_path = file_path + k + "/" 
                
            inum = sum(os.path.isfile(os.path.join(image_path, name)) for name in os.listdir(image_path))
            
            size = self.Minsize(image_path)
            pnum = size * size
            allarray = np.empty((inum, pnum))
            ave = np.empty(pnum)
            
            pixel = 0

            for i, image in enumerate(os.listdir(image_path)):
                
                img = cv2.imread(image_path + image, cv2.IMREAD_GRAYSCALE)
                allarray[i, :]  =  img.reshape(-1)

            for i in range(pnum):
                for j in range(inum):
                    pixel += allarray[j,i]

                pixel //= inum
                ave[i] = pixel

            array = np.reshape(ave, (size, size))
            os.makedirs(ave_file_name, exist_ok = True)

            if extension[1] == ".jpg" or extension[1] == ".png":
                cv2.imwrite(ave_file_name + "/"  + extension[0] + '.jpg', array)
                break
            else:
                cv2.imwrite(ave_file_name + "/"  + k + '.jpg', array)
                
        messagebox.showinfo('報告', '生成完了しました')
        
# ──────────── 最短距離法確率算出 ──────────── #
    # ランダムに100枚選出
    def Random(self, memPath, tesPath):

        name_list = glob.glob(memPath + "*")

        while True:

            num = sum(os.path.isfile(os.path.join(tesPath, name)) for name in os.listdir(tesPath))

            if num == 100:
                break

            name = random.choice(name_list)
            shutil.copy(name, tesPath)
            
    # ハスを取得する関数
    def Path_get(self):
        
        # 検証フォルダ名
        newPath = self.entry3_frame_pro.get() + "/"
        # 検証画像パス
        memPath = self.entry2_frame_pro.get() + "/"
        # 平均画像パス
        avePath = self.entry1_frame_pro.get() + "/"
        # 検証回数
        num = self.entry4_frame_pro.get()
        
        # 自動生成
        ansPath = "./Ans/"
        tesPath = "./inspection/"
        
        for i in os.listdir(avePath):
            create = ansPath + os.path.splitext(os.path.basename(i))[0]
            os.makedirs(create, exist_ok=True)
            
        os.makedirs(tesPath, exist_ok=True)
        
        for i in memPath, avePath:
            self.Check_path(i)
            
        self.Check_name(newPath)
        self.Check_num(num)
        
        for i in range(int(num)):
            self.Probability(newPath, memPath, tesPath, avePath, ansPath, i)
        
        messagebox.showinfo('報告', '算出完了しました')
    
    # 確率算出
    def Probability(self, newPath, memPath, tesPath, avePath, ansPath, i):
        
        # 結果出力テキスト用ファイルの生成 & パスの準備
        wirPath = newPath + str(i + 1)+ "th/"
        os.makedirs(wirPath, exist_ok=True)

        # 画像100枚選出
        self.Random(memPath, tesPath)

        # ファイル数取得
        anum = sum(os.path.isfile(os.path.join(avePath, name)) for name in os.listdir(avePath))
        tnum = sum(os.path.isfile(os.path.join(tesPath, name)) for name in os.listdir(tesPath))

        # 画像サイズ取得
        size = self.Minsize(avePath)

        # 総画素数計算
        pnum = size * size

        # 配列
        ans = np.empty((anum,tnum,pnum))
        aarray = np.empty((anum,pnum))
        tarray = np.empty((tnum,pnum))
        result = np.zeros(anum * anum)
        final = np.zeros((tnum,anum))
        mini = np.zeros(tnum)
        index = np.zeros(2)

        # 変数宣言
        diff = 0
        count = 0
        count2 = 0
        absdiff = 0
        horizontal = 0
        correctAns = 0

        # 選別ファイルの初期化
        for i in os.listdir(ansPath):

            shutil.rmtree(ansPath + i)
            os.mkdir(ansPath + i)

        # 平均画像の格納
        for i, name in enumerate(os.listdir(avePath)):
            
            img = cv2.imread(avePath + name,cv2.IMREAD_GRAYSCALE)
            aarray[i, :] = img.reshape(-1)
            
        # 検証画像の格納
        for i, name in enumerate(os.listdir(tesPath)):
            img = cv2.imread(tesPath + name,cv2.IMREAD_GRAYSCALE)
            tarray[i, :] = img.reshape(-1)

        # 計算
        for i in range(pnum):
            for j in range(anum):
                for k in range(tnum):

                    diff = aarray[j,i] - tarray[k,i]
                    ans[j,k,i] = np.abs(diff)

        # 結果
        for j in range(anum):
            for k in range(tnum):
                for i in range(pnum):

                    final[k,j] += ans[j,k,i]
        # 最小値
        for k in range(tnum):
            mini[k] = final[k,0]

            for j in range(anum):

                if mini[k] > final[k,j]:
                    mini[k] = final[k,j]

        # 最小値から画像を特定
        for i,file in enumerate(os.listdir(tesPath)):

            dex = np.argwhere(final == mini[i])
            index = np.ravel(dex)

            for j in os.listdir(ansPath):
                count += 1
                
                if count == index[1] + 1:
                    
                    shutil.move(tesPath + file, ansPath + j)
                    count = 0
                    
                    break

        # 結果を書き込み
        for k in os.listdir(ansPath):

            txtPath = ansPath + k
            t = open(wirPath + k +".txt","w")

            for i in os.listdir(ansPath):
                for j in os.listdir(txtPath):

                    sent = re.split("\d+",j)

                    if i == sent[0]:
                        count2 += 1

                if count2 != 0:
                    t.write(i + " " + str(count2) + "\n")

                result[horizontal] = count2

                count2 = 0
                horizontal += 1
                
        array = np.reshape(result,(anum,anum))

        # 確率の算出
        for i in range(anum):
            correctAns += array[i,i]

        t = open(wirPath + str(correctAns) +"%.txt","w")
        t.close()
        
# 始まり
if __name__=="__main__":
    
    root = tk.Tk()
    try:
        app = Application(master = root)#Inherit
    except:
        messageboxshowerror("予期せぬエラー", "使い方を再度確認してください")
    app.mainloop()